<h1 style="color: green;">Considerations for time series</h1>
<p>
Data leakage is the introduction of information about the target of a data mining problem,<br>
which should not be legitimately available to mine from.<br>


Data leakage is the creation of unexpected additional information in the training data<br>
allowing a model or machine learning algorithm to make unrealistically good predictions.<br>


Data leakage is the unintentional introduction of predictive information about the target<br>
by the data collection, aggregation and preparation process.<br>

In processing data for the classifier model, the data has not been split to account for<br>
the data timestamp i.e. the data is randomly sampled so there is relatively future data<br>
points in training and testing datasets. This is called data leakage. However since<br>
time series has suspicious performance, it is waved:<br><br>
<strong>
Michael Burry did not have a time series/forcasting model that predicted an imenent<br>
stock market crash in 2008. Instead, Michael stumbled onto data which indicated<br>
a serious problem in the subprime lending, the rest is common sense.</strong><br><br>
<strong>
Back in 2006-2007 no financial forcasting models successfully predicted the crash in 2008:<br> 
they didn't have the data that indicated a serious problem in the underlying assets, subprime lending.</strong><br><br>
<strong>
Forecasting models do well when there is data supporting the trend:<br>
you can see a wave building by the sea side and follow it until it collapses but you can't<br>
predict where and when it will collapse with certainty. Nore where and when the next one will arise<br>
with certainty unless you have multiple detectors beneath the waters.</strong><br><br>
<strong>
But you know in advance that one will arise, eventually.</strong><br><br>
<strong>
So the question, is time series real data science?</strong><br>



</p>